In [12]:
# set up working catalog
import sys
from pathlib import Path
project_path = str(Path().cwd().parent.resolve())
if project_path not in sys.path:
    sys.path.append(project_path)

# imports
from common.utils import *

import warnings
import numpy as np
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.discriminant_analysis import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from common.custom_logistic_regression import CustomLogisticRegressionMulticlass
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [13]:
data = get_data()

X = data.drop("Target", axis=1)
y = data["Target"]

encoder = LabelEncoder()
y = encoder.fit_transform(y)

X_train, X_val, X_test, y_train, y_val, y_test = split(X, y)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), X.select_dtypes(include=["object"]).columns),
        ("num", StandardScaler(), X.select_dtypes(include=["int64", "float64"]).columns.tolist())
    ]
)

data.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,single,2nd phase - general contingent,6th choice,Animation and Multimedia Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 3rd Cycle,Other - 11th Year,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,single,International student (bachelor),2nd choice,Tourism,Daytime,Secondary education,160.0,Portuguese,Secondary Education - 12th Year or Eq.,Higher Ed - Degree,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,single,1st phase - general contingent,6th choice,Communication Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 1st Cycle (4th/5th),...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,single,2nd phase - general contingent,3rd choice,Journalism and Communication,Daytime,Secondary education,122.0,Portuguese,Basic Ed 2nd Cycle (6th–8th),Basic Ed 1st Cycle (4th/5th),...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,married,Over 23 years old,2nd choice,Social Service (evening attendance),Evening,Secondary education,100.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 2nd Cycle (6th–8th),...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [14]:
get_accurancy_dataframe(LogisticRegression(max_iter=200), preprocessor, X_train,
                        y_train, X_test, y_test)

,0,1,2,3,4
Train,0.814620,0.820679,0.825121,0.814216,0.812197
Validation,0.766129,0.748387,0.730645,0.766129,0.785484
Test,0.754078,0.750314,0.754078,0.755332,0.742785


In [15]:
warnings.filterwarnings("ignore", category=FutureWarning)

model = CustomLogisticRegressionMulticlass(epochs=600, batch_size=96, learning_rate=0.01)
result = get_accurancy_dataframe(model, preprocessor, X_train,
                        y_train, X_test, y_test, cv=3)

result

,0,1,2
Train,0.792407,0.794830,0.790388
Validation,0.780645,0.751613,0.754839
Test,0.756587,0.754078,0.749059


In [16]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(max_iter=200))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.72      0.74       253
           1       0.58      0.39      0.47       152
           2       0.79      0.92      0.85       392

    accuracy                           0.76       797
   macro avg       0.71      0.68      0.69       797
weighted avg       0.74      0.76      0.74       797



In [17]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", CustomLogisticRegressionMulticlass(epochs=600, batch_size=96, learning_rate=0.01))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.73      0.76       253
           1       0.62      0.32      0.42       152
           2       0.76      0.95      0.84       392

    accuracy                           0.76       797
   macro avg       0.73      0.66      0.68       797
weighted avg       0.75      0.76      0.74       797

